In [90]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch.nn.functional as F

In [212]:
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

In [137]:
import pandas as pd

In [174]:
def classify_sentiment_detail(text) :
    # 문장을 토큰화하고 텐서로 변환
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # 모델을 사용하여 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)

    # 예측 결과 해석
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)  # 각 레이블에 대한 확률 계산
    predicted_class_id = torch.argmax(probabilities, dim=1).item()

    # 레이블 ID와 레이블 이름을 매핑
    label_map = model.config.id2label
    predicted_label = label_map[predicted_class_id]
    predicted_score = probabilities[0][predicted_class_id].item()

    # 모든 레이블에 대한 점수 출력
    all_scores = {label_map[i]: probabilities[0][i].item() for i in range(len(label_map))}
    
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {predicted_label} (Score: {predicted_score:.4f})")
    print("All Scores:", all_scores)

In [179]:
def get_sentiment(text) :
    # 문장을 토큰화하고 텐서로 변환
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # 모델을 사용하여 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)

    # 예측 결과 해석
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)  # 각 레이블에 대한 확률 계산
    predicted_class_id = torch.argmax(probabilities, dim=1).item()

    # 레이블 ID와 레이블 이름을 매핑
    label_map = model.config.id2label
    predicted_label = label_map[predicted_class_id]
    #predicted_score = probabilities[0][predicted_class_id].item()
    predicted_score = probabilities[0][1].item()

    # 모든 레이블에 대한 점수 출력
    #all_scores = {label_map[i]: probabilities[0][i].item() for i in range(len(label_map))}
    
    #print(f"Text: {text}")
    #print(f"Predicted Sentiment: {predicted_label} (Score: {predicted_score:.4f})")
    #print("All Scores:", all_scores)
    
    return predicted_score

In [151]:
def classify_sentiment(text) :
    # 문장을 토큰화하고 텐서로 변환
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # 모델을 사용하여 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)

    # 예측 결과 해석
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)  # 각 레이블에 대한 확률 계산
    predicted_class_id = torch.argmax(probabilities, dim=1).item()

    # 레이블 ID와 레이블 이름을 매핑
    label_map = model.config.id2label
    predicted_label = label_map[predicted_class_id]
    #predicted_score = probabilities[0][predicted_class_id].item()

    # 모든 레이블에 대한 점수 출력
    #all_scores = {label_map[i]: probabilities[0][i].item() for i in range(len(label_map))}
    
    #print(f"Text: {text}")
    #print(f"Predicted Sentiment: {predicted_label} (Score: {predicted_score:.4f})")
    #print("All Scores:", all_scores)
    
    return predicted_label

In [218]:
text = '안녕하세요. 또 만났군요.'

In [223]:
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

In [224]:
with torch.no_grad():
        outputs = model(**inputs)

In [225]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0338, -0.0028]], device='cuda:0'), hidden_states=None, attentions=None)

In [226]:
logits = outputs.logits

In [227]:
probabilities = F.softmax(logits, dim=1)  # 각 레이블에 대한 확률 계산

In [228]:
probabilities

tensor([[0.4923, 0.5077]], device='cuda:0')

In [229]:
torch.argmax(probabilities, dim=1).item()

1

In [192]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [125]:
model.config.id2label

{0: '0', 1: '1'}

In [2]:
tokenizer = AutoTokenizer.from_pretrained('/home/osung/models/huggingface/sentiment')

In [3]:
model = AutoModelForSequenceClassification.from_pretrained('/home/osung/models/huggingface/sentiment')

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at /home/osung/models/huggingface/sentiment and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
print(model)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [217]:
print(model.config)

ElectraConfig {
  "_name_or_path": "/home/osung/models/huggingface/sentiment",
  "architectures": [
    "ElectraModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 35000
}



In [127]:
classify_sentiment('완벽해요')

Text: 완벽해요
Predicted Sentiment: 1 (Score: 0.5388)
All Scores: {'0': 0.4612140655517578, '1': 0.5387859344482422}


In [129]:
text = "“교육·교통·문화 모두 갖춘 강남 최고의 입지” 지상 38층·지하 3층에 15개동 1,612세대 중 일반분양 213세대 한강 조망권·주거환경 등 호평 모델하우스 수요자 발길 줄이어 9·1 부동산대책 발표 후 강남 재건축단지의 몸값이 나날이 높아지고 있는 가운데 서울 강남권 재건축 아파트 중 신반포1차를 재건축한 `아크로리버 파크 2회차'가 9·1 대책 발표 후 가장 먼저 분양에 나서면서 고액 자산가들의 관심이 집중되고 있다. 강남 최고 부촌 아파트를 넘보는 대림산업의 `아크로리버 파크 2회차'가 지난 19일 견본주택의 문을 열고 본격 분양에 들어갔다. 단지는 지하 3층~지상 38층, 15개 동, 전용면적 59~164㎡로 전체 1,612세대 중 213세대가 일반 분양된다. 세부면적을 살펴보면, △59㎡ 40세대 △84㎡ 118세대 △112㎡ 16세대 △129㎡ 33세대 △164㎡ 6세대로 중소형이 74%를 차지한다. 입주시기는 2016년 8월 예정이다. ■반포지구 재건축단지 중 최고 층수로 지어져 … 반포지역 프리미엄 톡톡=반포 `아크로리버 파크'는 교육, 교통, 문화, 편의시설 등 완벽한 주거환경을 갖춰 대표 부촌으로 자리를 굳히고 있는 반포의 중심에 위치하는 만큼 지역 프리미엄을 톡톡히 누릴 수 있는 곳이다. 또한 이 단지는 강남 한강변에 10년 만에 공급된 아파트인데다 강남 한강변에서는 최초로 특별건축구역으로 지정돼 최고 38층 초고층으로 건설된다. 다른 반포지구 재건축단지 최고 층수는 34~35층으로 향후 반포동 랜드마크 단지로서의 역할이 기대된다. ■한강 조망 가능한 `고층 중심', 대리석 마감 및 독일산 주방가구로 `상품 업그레이드'=아크로리버 파크 2회차의 분양가는 현재 서초구에서 가장 고가 아파트이며 입주 5년차를 맞은 `래미안퍼스티지'의 전용 59㎡가 8억9,000만~10억5,000만원 사이에 거래되는 것과 비교해 비슷한 수준에 책정됐다. 아크로리버파크 2회차 총 분양가는 전용 59㎡가 8억4,900만~10억5,000만원, 전용 84㎡는 11억8,000만~15억4,500만원선으로 책정됐다. 전용 112㎡는 16억3,000만~20억1,000만원, 전용 129㎡ 18억~21억4,000만원, 전용 164㎡는 21억8,000만~23억9,000만원이다. 계약조건은 계약금 10%, 중도금 이자 후불제가 적용된다. ■최고 부촌에 지어지는 최고가 명품 아파트답게 고급화된 설계 눈길=반포 `아크로리버 파크'가 강남 노른자위에 지어지는 최고가 명품 단지인 만큼 2회차 물량에도 최고급 설계가 적용된다. 우선 동 배치를 오픈 뷰(Open view)와 59m의 동 간 이격 거리로 실현해 탁 트인 조망권과 풍부한 일조권을 확보했다. 또한 단지 내 6개 테마정원뿐만 아니라 서측공원과 보행로를 따라 한강시민공원 반포지구까지 연결돼 있어 쾌적한 주거환경을 갖췄으며, 단지 내에서도 메인 커뮤니티 및 지하철 접근성이 우수한 것도 특징으로 꼽힌다. 견본주택은 서울 강남구 신사동 631 도산공원 사거리에서 성수대교 남단 교차로 방면 우측에 있다."

In [130]:
len(text)

1492

In [131]:
classify_sentiment(text)

Text: “교육·교통·문화 모두 갖춘 강남 최고의 입지” 지상 38층·지하 3층에 15개동 1,612세대 중 일반분양 213세대 한강 조망권·주거환경 등 호평 모델하우스 수요자 발길 줄이어 9·1 부동산대책 발표 후 강남 재건축단지의 몸값이 나날이 높아지고 있는 가운데 서울 강남권 재건축 아파트 중 신반포1차를 재건축한 `아크로리버 파크 2회차'가 9·1 대책 발표 후 가장 먼저 분양에 나서면서 고액 자산가들의 관심이 집중되고 있다. 강남 최고 부촌 아파트를 넘보는 대림산업의 `아크로리버 파크 2회차'가 지난 19일 견본주택의 문을 열고 본격 분양에 들어갔다. 단지는 지하 3층~지상 38층, 15개 동, 전용면적 59~164㎡로 전체 1,612세대 중 213세대가 일반 분양된다. 세부면적을 살펴보면, △59㎡ 40세대 △84㎡ 118세대 △112㎡ 16세대 △129㎡ 33세대 △164㎡ 6세대로 중소형이 74%를 차지한다. 입주시기는 2016년 8월 예정이다. ■반포지구 재건축단지 중 최고 층수로 지어져 … 반포지역 프리미엄 톡톡=반포 `아크로리버 파크'는 교육, 교통, 문화, 편의시설 등 완벽한 주거환경을 갖춰 대표 부촌으로 자리를 굳히고 있는 반포의 중심에 위치하는 만큼 지역 프리미엄을 톡톡히 누릴 수 있는 곳이다. 또한 이 단지는 강남 한강변에 10년 만에 공급된 아파트인데다 강남 한강변에서는 최초로 특별건축구역으로 지정돼 최고 38층 초고층으로 건설된다. 다른 반포지구 재건축단지 최고 층수는 34~35층으로 향후 반포동 랜드마크 단지로서의 역할이 기대된다. ■한강 조망 가능한 `고층 중심', 대리석 마감 및 독일산 주방가구로 `상품 업그레이드'=아크로리버 파크 2회차의 분양가는 현재 서초구에서 가장 고가 아파트이며 입주 5년차를 맞은 `래미안퍼스티지'의 전용 59㎡가 8억9,000만~10억5,000만원 사이에 거래되는 것과 비교해 비슷한 수준에 책정됐다. 아크로리버파크 2회차 총 분양가는 전용 59㎡가 8억4,900만~10억5,000만원, 전용 84㎡

In [132]:
text = '허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다. 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다. 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다.'

In [133]:
len(text)

274

In [134]:
classify_sentiment(text)

Text: 허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다. 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다. 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다.
Predicted Sentiment: 0 (Score: 0.5309)
All Scores: {'0': 0.5309069752693176, '1': 0.46909299492836}


In [135]:
text = "문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 우리가 한미간에 합의를 한 것이 이번 발사 이전인데, 그 이후에 또 다시 발사가 있었기 때문에 이 점에 대해서는 국민들의 공감이나 제재가 필요하다고 생각한다. 한일관계는 굉장히 중요하다고 생각하고, 앞으로 더 미래지향적으로 발전돼 나가야 한다고 생각한다.고령화가 급격하게 진행되고 있어서 65세 이상 인구가 14%가 넘는 고령사회를 이미 2017년에 통과했고, 2025년이면 20%가 넘는 초고령사회가 될 것으로 예상된다. 고령인구가 크게 늘고 있기 때문에 일자리 수를 더 늘리고 과거의 급여가 낮았기 때문에 급여를 두배 높여서 실제로 어르신들의 빈곤 해결에 도움되도록 정부가 노력하고 있는 것이다. 거시적인 경제성공은 인정하고 자부심을 가져야 한다고 생각한다. 다만 국민들에게 고르게 다 소득배분이 되고 있지 않고 있기 때문에 아직도 양극화가 심각하고 특히 소득이 낮은 층의 소득이 늘지 않고 있기때문에 그분들의 사는 문제가 해결이 안 되고 있다."

In [136]:
classify_sentiment(text)

Text: 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 

In [140]:
test_df = pd.read_csv('/home/osung/data/korean/finance/finance_data_test.csv')

In [141]:
test_df

,labels,text
0,0.5,이 회사는 러시아의 톰스크 지역에 있는 베니어 공장에 기계를 납품하기로 되어 있었다.
1,0.5,UNC 샬롯은 또한 SSH Tetia Connector를 배포하여 안전한 응용 프로...
2,0.5,"2009년에 리앤맨은 연간 450만 톤의 종이와 300,000 톤의 펄프를 생산할 ..."
3,0.5,`` 그러나 핀란드 관광객들이 구매한 맥주도 포함돼 있다는 점을 상기하며 유럽 규모...
4,0.5,"핀란드에서는 알마미디어의 Etuovi.com이 해당 서비스를 제공하고 있는데, 이는..."
...,...,...
1449,0.5,2006년과 2007년의 총 투자액은 약 75만 유로에 달할 것으로 예상됩니다.
1450,0.5,그 확장된 회사는 계속해서 NewPage로 불릴 것이다.
1451,0.5,금요일에 활성화되도록 설정된 웜은 '를 포함하여 가장 일반적인 파일 형식을 사용하여...
1452,0.5,장비는 핀란드 홀롤라에 있는 Vahto의 작업장에서 생산될 예정이며 2009년 1/...


In [180]:
results = []

for text in test_df.text:
    result = get_sentiment(text)
    results.append(result)
    
test_df['result'] = results

In [144]:
get_sentiment(test_df.iloc[0].text)

0.5164045691490173

In [146]:
test_df

,labels,text,result
0,0.5,이 회사는 러시아의 톰스크 지역에 있는 베니어 공장에 기계를 납품하기로 되어 있었다.,0.516405
1,0.5,UNC 샬롯은 또한 SSH Tetia Connector를 배포하여 안전한 응용 프로...,0.522234
2,0.5,"2009년에 리앤맨은 연간 450만 톤의 종이와 300,000 톤의 펄프를 생산할 ...",0.519056
3,0.5,`` 그러나 핀란드 관광객들이 구매한 맥주도 포함돼 있다는 점을 상기하며 유럽 규모...,0.515279
4,0.5,"핀란드에서는 알마미디어의 Etuovi.com이 해당 서비스를 제공하고 있는데, 이는...",0.522005
...,...,...,...
1449,0.5,2006년과 2007년의 총 투자액은 약 75만 유로에 달할 것으로 예상됩니다.,0.529533
1450,0.5,그 확장된 회사는 계속해서 NewPage로 불릴 것이다.,0.521242
1451,0.5,금요일에 활성화되도록 설정된 웜은 '를 포함하여 가장 일반적인 파일 형식을 사용하여...,0.520972
1452,0.5,장비는 핀란드 홀롤라에 있는 Vahto의 작업장에서 생산될 예정이며 2009년 1/...,0.507534


In [147]:
test_df['diff'] = test_df['labels'] - test_df['result']

In [148]:
test_df['diff']

0      -0.016405
1      -0.022234
2      -0.019056
3      -0.015279
4      -0.022005
          ...   
1449   -0.029533
1450   -0.021242
1451   -0.020972
1452   -0.007534
1453    0.479879
Name: diff, Length: 1454, dtype: float64

In [149]:
test_df['diff'].describe()

count    1454.000000
mean        0.072133
std         0.310558
min        -0.529161
25%        -0.018837
50%        -0.008421
75%         0.474849
max         0.499910
Name: diff, dtype: float64

In [150]:
test_df['result'].describe()

count    1454.000000
mean        0.513493
std         0.008031
min         0.500023
25%         0.506944
50%         0.512960
75%         0.519241
max         0.536858
Name: result, dtype: float64

In [163]:
results = []

for text in test_df.text:
    result = int(classify_sentiment(text))
    results.append(result)
    
test_df['class'] = results

In [181]:
test_df

,labels,text,result,diff,class,diff2
0,0.5,이 회사는 러시아의 톰스크 지역에 있는 베니어 공장에 기계를 납품하기로 되어 있었다.,0.516405,-0.016405,1,0.5
1,0.5,UNC 샬롯은 또한 SSH Tetia Connector를 배포하여 안전한 응용 프로...,0.522234,-0.022234,1,0.5
2,0.5,"2009년에 리앤맨은 연간 450만 톤의 종이와 300,000 톤의 펄프를 생산할 ...",0.519056,-0.019056,1,0.5
3,0.5,`` 그러나 핀란드 관광객들이 구매한 맥주도 포함돼 있다는 점을 상기하며 유럽 규모...,0.515279,-0.015279,1,0.5
4,0.5,"핀란드에서는 알마미디어의 Etuovi.com이 해당 서비스를 제공하고 있는데, 이는...",0.522005,-0.022005,1,0.5
...,...,...,...,...,...,...
1449,0.5,2006년과 2007년의 총 투자액은 약 75만 유로에 달할 것으로 예상됩니다.,0.529533,-0.029533,1,0.5
1450,0.5,그 확장된 회사는 계속해서 NewPage로 불릴 것이다.,0.521242,-0.021242,1,0.5
1451,0.5,금요일에 활성화되도록 설정된 웜은 '를 포함하여 가장 일반적인 파일 형식을 사용하여...,0.479028,-0.020972,0,-0.5
1452,0.5,장비는 핀란드 홀롤라에 있는 Vahto의 작업장에서 생산될 예정이며 2009년 1/...,0.507534,-0.007534,1,0.5


In [165]:
test_df['class'].value_counts()

1    1032
0     422
Name: class, dtype: int64

In [166]:
test_df['diff2'] = test_df['class'] - test_df['labels']

In [167]:
test_df['diff2'].describe()

count    1454.000000
mean        0.124140
std         0.434524
min        -1.000000
25%         0.000000
50%         0.000000
75%         0.500000
max         1.000000
Name: diff2, dtype: float64

In [168]:
test_df['diff2']

0       0.5
1       0.5
2       0.5
3       0.5
4       0.5
       ... 
1449    0.5
1450    0.5
1451   -0.5
1452    0.5
1453    0.0
Name: diff2, Length: 1454, dtype: float64

In [182]:
test_df[:20]

,labels,text,result,diff,class,diff2
0,0.5,이 회사는 러시아의 톰스크 지역에 있는 베니어 공장에 기계를 납품하기로 되어 있었다.,0.516405,-0.016405,1,0.5
1,0.5,UNC 샬롯은 또한 SSH Tetia Connector를 배포하여 안전한 응용 프로...,0.522234,-0.022234,1,0.5
2,0.5,"2009년에 리앤맨은 연간 450만 톤의 종이와 300,000 톤의 펄프를 생산할 ...",0.519056,-0.019056,1,0.5
3,0.5,`` 그러나 핀란드 관광객들이 구매한 맥주도 포함돼 있다는 점을 상기하며 유럽 규모...,0.515279,-0.015279,1,0.5
4,0.5,"핀란드에서는 알마미디어의 Etuovi.com이 해당 서비스를 제공하고 있는데, 이는...",0.522005,-0.022005,1,0.5
5,0.5,공사는 2007년 4월부터 6월까지 시작되어 2008년 초에 완공될 예정이다.,0.506076,-0.006076,1,0.5
6,1.0,핀란드 소유의 전자제품 계약 제조업체인 Elcoteq Hungary Kft는 회사가...,0.513823,0.486177,1,0.0
7,0.5,"이노바 2 프로젝트의 총 면적은 약 10,000 sq m (107,600 sq ft...",0.512156,-0.012156,1,0.5
8,0.5,"핀란드의 가정 장식 및 예술 공예 소매업체 티이마리(Tiimari Plc, OMX ...",0.482384,-0.017616,0,-0.5
9,0.5,"납품은 2007년 하반기에 시작될 것이며, 제분소의 시작은 2008년에 예정되어 있다.",0.483860,-0.016140,0,-0.5


In [178]:
classify_sentiment_detail(test_df.iloc[8].text)

Text: 핀란드의 가정 장식 및 예술 공예 소매업체 티이마리(Tiimari Plc, OMX 헬싱키: TII1V)는 월요일 (8월 18일) 2008년 1월부터 6월까지의 순매출액 33.9m로 3.3m의 영업손실을 기록했다고 보고했다.
Predicted Sentiment: 0 (Score: 0.5176)
All Scores: {'0': 0.5176163911819458, '1': 0.4823836386203766}


In [183]:
test_df['length'] = test_df['text'].apply(len)

In [184]:
test_df

,labels,text,result,diff,class,diff2,length
0,0.5,이 회사는 러시아의 톰스크 지역에 있는 베니어 공장에 기계를 납품하기로 되어 있었다.,0.516405,-0.016405,1,0.5,47
1,0.5,UNC 샬롯은 또한 SSH Tetia Connector를 배포하여 안전한 응용 프로...,0.522234,-0.022234,1,0.5,62
2,0.5,"2009년에 리앤맨은 연간 450만 톤의 종이와 300,000 톤의 펄프를 생산할 ...",0.519056,-0.019056,1,0.5,52
3,0.5,`` 그러나 핀란드 관광객들이 구매한 맥주도 포함돼 있다는 점을 상기하며 유럽 규모...,0.515279,-0.015279,1,0.5,64
4,0.5,"핀란드에서는 알마미디어의 Etuovi.com이 해당 서비스를 제공하고 있는데, 이는...",0.522005,-0.022005,1,0.5,96
...,...,...,...,...,...,...,...
1449,0.5,2006년과 2007년의 총 투자액은 약 75만 유로에 달할 것으로 예상됩니다.,0.529533,-0.029533,1,0.5,44
1450,0.5,그 확장된 회사는 계속해서 NewPage로 불릴 것이다.,0.521242,-0.021242,1,0.5,31
1451,0.5,금요일에 활성화되도록 설정된 웜은 '를 포함하여 가장 일반적인 파일 형식을 사용하여...,0.479028,-0.020972,0,-0.5,59
1452,0.5,장비는 핀란드 홀롤라에 있는 Vahto의 작업장에서 생산될 예정이며 2009년 1/...,0.507534,-0.007534,1,0.5,63


In [185]:
test_df['length'].describe()

count    1454.000000
mean       60.698074
std        28.355093
min         6.000000
25%        39.000000
50%        55.000000
75%        76.000000
max       169.000000
Name: length, dtype: float64

In [188]:
test_df.sort_values(by='length', ascending=False)

,labels,text,result,diff,class,diff2,length
768,0.5,Proha Plc (Euronext :7327)는 오늘 (5월 19일) 완전 소유 ...,0.499300,-0.000700,0,-0.5,169
940,1.0,(ADP News ) - 2008년 11월 3일 - 핀란드 화물 처리 시스템 및 서...,0.505610,0.494390,1,0.0,164
1328,0.5,기업 IT 업데이트 - ( C) 1995-2009 M2 Communications ...,0.504175,-0.004175,1,0.5,160
839,0.5,노르딕 비즈니스 보고서-2006년 1월 17일-에포어 플럭스는 에스토니아에 새로운 ...,0.503659,-0.003659,1,0.5,158
707,1.0,KAUKO-TELKO LTD 보도 자료 19.06.27 14.00 1 ( 1) Te...,0.487515,0.487515,0,-1.0,157
...,...,...,...,...,...,...,...
1094,0.5,리오라가 새 가방을 가졌어,0.513551,-0.013551,1,0.5,14
1258,0.5,누구나 환영합니다.,0.534867,-0.034867,1,0.5,10
1059,0.5,넌 혼자가 아니야,0.504075,-0.004075,1,0.5,9
510,0.5,NWC 분석:,0.496722,-0.003278,0,-0.5,7


In [190]:
test_df.loc[768].text

'Proha Plc (Euronext :7327)는 오늘 (5월 19일) 완전 소유 자회사인 Safran Software Solutions AS가 노르웨이에 본사를 둔 투자 그룹인 SNA Holding AS에 49%의 Safran North America LLC 지분을 매각하기로 합의했다고 발표했습니다.'

In [191]:
test_df.loc[707].text

'KAUKO-TELKO LTD 보도 자료 19.06.27 14.00 1 ( 1) Telko Group(Kauko-Telko)은 BP의 스웨덴 소매 회사인 Molub-Aloy AB의 총 소유권을 인수하여 BP 산업 윤활유 및 서비스(BP ILS)와 전략적 파트너십을 확장하고 있습니다.'

In [193]:
train_df = pd.read_csv('/home/osung/data/korean/finance/finance_data_train.csv')

In [194]:
train_df

,labels,text
0,0.0,2009년 영업이익은 이전에 발표된 전망보다 낮다.
1,0.5,레흐덴테키자트의 순매출은 2007년 약 1400만 유로였으며 70명의 직원을 두고 있다.
2,0.5,지역 사회 정원 가꾸기는 특히 채소를 기르기 위한 인기 있는 활동이 되었다.
3,0.5,"알파 그룹은 신설 회사의 43.9%, 텔레노르는 35.4%의 자유 유동 주식을 보유..."
4,0.5,이 중 38.8 유로가 순이자 수익이었다.
...,...,...
3387,0.0,"통화효과는 3.0pct, 즉 20mln 유로($31.3mln)로 수익에 부정적인 영..."
3388,1.0,"`` 리드스코에 피보의 투자 프로그램은 생산의 통합 현대화, 운송 및 물류 인프라 ..."
3389,0.5,"제품에는 휴대폰과 같은 가전 기기, 셋톱 박스, 평면 패널 TV뿐만 아니라 인프라 ..."
3390,0.5,이 다리는 14번 고속도로 개발 프로젝트의 일부이다.


In [197]:
model = model.to(device)

In [198]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        attention_mask = self.attention_masks[idx]
        label = self.labels[idx]

        return input_id, attention_mask, label

    def __len__(self):
        return len(self.input_ids)

In [200]:
def get_encode_data(tokenizer, sentences, labels, max_length=512):
    encoded_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=max_length)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

In [201]:
train_input_ids, train_attention_masks, train_labels = get_encode_data(tokenizer, train_df['text'].tolist(), train_df['labels'], max_length=512)

In [202]:
train_dataset = TrainDataset(train_input_ids, train_attention_masks, train_labels)

In [204]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

In [207]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [208]:
num_epochs = 10

In [209]:
num_training_steps = num_epochs * len(train_dataloader)

In [213]:
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


In [214]:
model.train()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [216]:
for epoch in range(0, num_epochs):
    epoch_loss = 0
    epoch_steps = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc=f'Epoch {epoch+1}', leave=False)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss.mean()
        logits = outputs.logits

        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        epoch_loss += loss.item()
        epoch_steps += 1

        if step % 1000 == 0:
            print(f'Epoch {epoch+1} / Step {step+1} - Loss: {epoch_loss/epoch_steps:.5f}')

    formatted_epoch = '_%02d' % epoch
    print('saved pth file :', pth_name+formatted_epoch)
    torch.save(model.state_dict(), pth_name+formatted_epoch) # save pth at every epoch

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   │                                                                                       │
│   10 │   │   model.zero_grad()                                                                   │
│   11 │   │                                                                                       │
│ ❱ 12 │   │   outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)          │
│   13 │   │                                                                                       │
│   14 │   │   loss = outputs.loss.mean()                                                          │
│   15 │   │   logits = outputs.logits                                                             │
│                                                                                                  │
│ /home/osung/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1501 in _call_impl  │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/osung/anaconda3/lib/python3.9/site-packages/transformers/models/electra/modeling_electra.p │
│ y:1032 in forward                                                                                │
│                                                                                                  │
│   1029 │   │   │   │   loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))        │
│   1030 │   │   │   elif self.config.problem_type == "multi_label_classification":                │
│   1031 │   │   │   │   loss_fct = BCEWithLogitsLoss()                                            │
│ ❱ 1032 │   │   │   │   loss = loss_fct(logits, labels)                                           │
│   1033 │   │                                                                                     │
│   1034 │   │   if not return_dict:                                                               │
│   1035 │   │   │   output = (logits,) + discriminator_hidden_states[1:]                          │
│                                                                                                  │
│ /home/osung/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1501 in _call_impl  │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks

In [232]:
device = torch.device("cpu")

In [233]:
model = model.to(device)

In [234]:
classify_sentiment_detail('이쁘고 좋아요~~~씻기도 편하고 아이고 이쁘다고 자기방에 갖다놓고 잘써요~^^')

Text: 이쁘고 좋아요~~~씻기도 편하고 아이고 이쁘다고 자기방에 갖다놓고 잘써요~^^
Predicted Sentiment: 1 (Score: 0.5292)
All Scores: {'0': 0.4708394706249237, '1': 0.5291605591773987}


In [235]:
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

In [236]:
review = '이쁘고 좋아요~~~씻기도 편하고 아이고 이쁘다고 자기방에 갖다놓고 잘써요~^^'

In [239]:
sentiment_classifier(review)

[{'label': '1', 'score': 0.5311794281005859}]

In [238]:
print(f'{review}\n>> {pred[0]}')

이쁘고 좋아요~~~씻기도 편하고 아이고 이쁘다고 자기방에 갖다놓고 잘써요~^^
>> {'label': '1', 'score': 0.49868595600128174}
